In [8]:
import pennylane as qml
from pennylane import numpy as np

n_register = 3
dev = qml.device('default.qubit', wires=(n_register), analytic=True)
def qft_rotations(n=0):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    n = int(n)
    if n == 0:
        return
    n -= 1
    qml.Hadamard(wires=n)
    for qubit in range(n):
        qml.CRZ(np.pi/2**(n-qubit), wires=[qubit, n])
    qft_rotations(n=n)

def swap_registers(n):
    for qubit in range(n//2):
        qml.SWAP(wires = [qubit, n-qubit-1])

def qft(n=0):
    """QFT on the first n qubits in circuit"""
    qft_rotations(n=n)
    swap_registers(n=n)
    return qml.probs(wires=[0, 1, 2])

circuit = qml.QNode(qft, dev)
circuit(n=3)
print(circuit.draw())

circuit(n=3)
print(qml.Hadamard(wires=[0]).diagonalizing_gates())

[RY(-0.7853981633974483, wires=[0])]

DeviceError: The analytic argument has been replaced by shots=None. Please use shots=None instead of analytic=True.

In [ ]:
>>> from pennylane.fourier.visualize import *
>>> import matplotlib.pyplot as plt

In [9]:
wires_m = [0, 1, 2]             # qubits needed to encode m
wires_k = [3, 4, 5]             # qubits needed to encode k
wires_solution = [6, 7, 8, 9]   # qubits needed to encode the solution

dev = qml.device("default.qubit", wires=wires_m + wires_k + wires_solution, shots=1)

n_wires = len(dev.wires) # total number of qubits used

def addition(wires_m, wires_k, wires_sol):
    # prepare solution qubits to counting
    qml.QFT(wires=wires_solution)

    # add m to the counter
    for i in range(len(wires_m)):
        qml.ctrl(add_k_fourier, control=wires_m[i])(2 **(len(wires_m) - i - 1), wires_solution)

    # add k to the counter
    for i in range(len(wires_k)):
        qml.ctrl(add_k_fourier, control=wires_k[i])(2 **(len(wires_k) - i - 1), wires_solution)

    # return to computational basis
    qml.adjoint(qml.QFT)(wires=wires_solution)

@qml.qnode(dev)
def sum2(m, k, wires_m, wires_k, wires_solution):
    # m and k codification
    qml.BasisEmbedding(m, wires=wires_m)
    qml.BasisEmbedding(k, wires=wires_k)

    # apply the addition circuit
    addition(wires_m, wires_k, wires_solution)

    return qml.sample(wires=wires_solution)

print(f"The ket representation of the sum of 7 and 3 is "
      f"{sum2(7, 3, wires_m, wires_k, wires_solution)}")

qml.draw_mpl(sum2, show_all_wires=True)(7, 3, wires_m, wires_k, wires_solution)
plt.show()


NameError: name 'add_k_fourier' is not defined

NameError: name 'QuantumCircuit' is not defined